In [7]:
import numpy as np
import cv2
import os
from matplotlib import pyplot as plt
from tqdm import tqdm

In [8]:
import tensorflow as tf

In [25]:
def normalize(x):
    normalizer = lambda y: cv2.normalize(y, y, 0, 255, cv2.NORM_MINMAX)  
    return np.array(list(map(normalizer,x)))
def reshape(x):
    return x.reshape(x.shape[0],-1)

In [26]:
def read_dataset(path):
    pictures = []
    lineplots = []
    
    resize = lambda x: cv2.resize(x,(256,256))
    
    for entry in tqdm(os.scandir(path+'/pictures/')):
        if (entry.is_file()):
            img = cv2.imread(entry.path,cv2.IMREAD_COLOR)
            pictures.append(resize(img))
    
    for entry in tqdm(os.scandir(path+'/lineplots/')):
        if (entry.is_file()):
            img = cv2.imread(entry.path,cv2.IMREAD_COLOR)
            lineplots.append(resize(img))
            
    return np.array(pictures), np.array(lineplots)

In [27]:
pictures, lineplots = read_dataset('dataset')

1673it [00:05, 299.32it/s]
182it [00:00, 1994.37it/s]


In [28]:
pictures.shape, lineplots.shape

((1673, 256, 256, 3), (182, 256, 256, 3))

In [29]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

In [30]:
normalize_transformer = FunctionTransformer(normalize)
reshape_transformer = FunctionTransformer(reshape)
steps = [('normalize',normalize_transformer),('reshape',reshape_transformer),('SVM',SVC())]
pipeline = Pipeline(steps)

In [31]:
def build_dataset(pictures,lineplots):
    x = np.concatenate((pictures,lineplots))
    y = np.array([0]*len(pictures) + [1]*len(lineplots))
    return x,y

In [32]:
x,y = build_dataset(pictures,lineplots)

In [33]:
x.shape, y.shape

((1855, 256, 256, 3), (1855,))

In [45]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [46]:
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('normalize',
                 FunctionTransformer(accept_sparse=False, check_inverse=True,
                                     func=<function normalize at 0x7ff00073a200>,
                                     inv_kw_args=None, inverse_func=None,
                                     kw_args=None, validate=False)),
                ('reshape',
                 FunctionTransformer(accept_sparse=False, check_inverse=True,
                                     func=<function reshape at 0x7ff00073a320>,
                                     inv_kw_args=None, inverse_func=None,
                                     kw_args=None, validate=False)),
                ('SVM',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='scale', kernel='rbf', max_iter=-1,
                     probability=False, random_state=None, shrinking=True,


In [50]:
from sklearn.metrics import precision_recall_fscore_support
y_pred = pipeline.predict(X_test)
precision_recall_fscore_support(y_test, y_pred)

(array([0.99802761, 0.98      ]),
 array([0.99802761, 0.98      ]),
 array([0.99802761, 0.98      ]),
 array([507,  50]))

creating a pickle:

In [52]:
import pickle
pickle_name = 'model.sav'
pickle.dump(pipeline,open(pickle_name, 'wb'))

checking the pickle:

In [54]:
loaded_model = pickle.load(open(pickle_name, 'rb'))

In [55]:
from sklearn.metrics import precision_recall_fscore_support
y_pred = loaded_model.predict(X_test)
precision_recall_fscore_support(y_test, y_pred)

(array([0.99802761, 0.98      ]),
 array([0.99802761, 0.98      ]),
 array([0.99802761, 0.98      ]),
 array([507,  50]))